In [1]:
import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *
from tqdm import tqdm_notebook 

In [2]:
path = 'C:/Users/choco/Desktop/Github/Dacon/Energy Bigdata Utilization/'
train = pd.read_csv(path + "input/train.csv")
test = pd.read_csv(path + "input/test.csv")

### 직전 시간의 전력 사용량으로 인한 결측 해결

https://dacon.io/cpt11/205743

In [3]:
for k in tqdm_notebook(range(1, len(train.columns))): #시간을 제외한 1열부터 마지막 열까지를 for문으로 작동시킵니다.
    train_median=train.iloc[:,k].median() #값을 대체하는 과정에서 값이 변경 될 것을 대비해 해당 세대의 중앙값을 미리 계산하고 시작합니다.
    counting=train.loc[ train.iloc[:,k].isnull()==False ][ train.columns[k] ].index

    df=pd.DataFrame( list( zip( counting[:-1], counting[1:] - counting[:-1] -1  ) ), columns=['index','count'] )

    df2= df[ (df['count'] > 0) ] #결측치가 존재하는 부분만 추출
    df2=df2.reset_index(drop=True) #기존에 존재하는 index를 초기화 하여 이후 for문에 사용함

    for i,j in zip( df2['index'], df2['count'] ) : # i = 해당 세대에서 값이 존재하는 index, j = 현재 index 밑의 결측치 갯수
        if train.iloc[i,k]>=train_median: #현재 index에 존재하는 값이 해당 세대의 중앙 값 이상일때만 분산처리 실행
            train.iloc[ i : i+j+1 , k] = train.iloc[i,k] / (j+1) 
            #현재 index 및 결측치의 갯수 만큼 지정을 하여, 현재 index에 있는 값을 해당 갯수만큼 나누어 줍니다
        else:  
            pass #현재 index에 존재하는 값이 중앙 값 미만이면 pass를 실행

In [4]:
for k in tqdm_notebook(range(1, len(test.columns))): #시간을 제외한 1열부터 마지막 열까지를 for문으로 작동시킵니다.
    test_median=test.iloc[:,k].median() #값을 대체하는 과정에서 값이 변경 될 것을 대비해 해당 세대의 중앙값을 미리 계산하고 시작합니다.
    counting=test.loc[ test.iloc[:,k].isnull()==False ][ test.columns[k] ].index

    df=pd.DataFrame( list( zip( counting[:-1], counting[1:] - counting[:-1] -1  ) ), columns=['index','count'] )

    df2= df[ (df['count'] > 0) ] #결측치가 존재하는 부분만 추출
    df2=df2.reset_index(drop=True) #기존에 존재하는 index를 초기화 하여 이후 for문에 사용함

    for i,j in zip( df2['index'], df2['count'] ) : # i = 해당 세대에서 값이 존재하는 index, j = 현재 index 밑의 결측치 갯수
        if test.iloc[i,k]>=test_median: #현재 index에 존재하는 값이 해당 세대의 중앙 값 이상일때만 분산처리 실행
            test.iloc[ i : i+j+1 , k] = test.iloc[i,k] / (j+1) 
            #현재 index 및 결측치의 갯수 만큼 지정을 하여, 현재 index에 있는 값을 해당 갯수만큼 나누어 줍니다
        else:  
            pass #현재 index에 존재하는 값이 중앙 값 미만이면 pass를 실행

In [5]:
train.to_csv(path + 'input/train2.csv',index=False) #결측치 대체 작업 이후 csv 파일 내보내기
test.to_csv(path + 'input/test2.csv',index=False) #결측치 대체 작업 이후 csv 파일 내보내기

### 시계열에 적합한 형태로 변환


In [6]:
train['Time'] = pd.to_datetime(train['Time']) 
train = train.set_index('Time')

test['Time'] = pd.to_datetime(test['Time']) 
test = test.set_index('Time')

In [ ]:
place_id=[]; time=[] ; target=[] # 빈 리스트를 생성합니다.
for i in tqdm_notebook(train.columns):
    for j in range(len(train)):
        place_id.append(i) # place_id에 미터 ID를 정리합니다.
        time.append(train.index[j]) # time에 시간대를 정리합니다.
        target.append(train[i].iloc[j]) # target에 전력량을 정리합니다.

new_df=pd.DataFrame({'place_id':place_id,'time':time,'target':target})
new_df=new_df.dropna() # 결측치를 제거합니다.
new_df=new_df.set_index('time') # time을 인덱스로 저장합니다.
new_df.to_csv(path + "input/new_df_train.csv",index=True)

In [ ]:
place_id=[]; time=[] ; target=[] # 빈 리스트를 생성합니다.
for i in tqdm_notebook(test.columns):
    for j in range(len(test)):
        place_id.append(i) # place_id에 미터 ID를 정리합니다.
        time.append(test.index[j]) # time에 시간대를 정리합니다.
        target.append(test[i].iloc[j]) # target에 전력량을 정리합니다.

new_df=pd.DataFrame({'place_id':place_id,'time':time,'target':target})
new_df=new_df.dropna() # 결측치를 제거합니다.
new_df=new_df.set_index('time') # time을 인덱스로 저장합니다.
new_df.to_csv(path + "input/new_df_test.csv",index=True)

### 정형데이터에 적합한 형태로 변환

In [ ]:
from tqdm import tqdm_notebook 

time = []
data = []
meter_id = []
for i in tqdm_notebook(np.arange(1, test2.shape[1])):
    index = test2[test2.iloc[0:, i].notnull() == True].index
    meter_id = np.append(meter_id, np.array([test2.columns[i]] * len(index)))
    time = np.append(time, test2.iloc[index, 0].values)
    data = np.append(data, test2.iloc[index, i].values)
    
test2_df = pd.DataFrame()
test2_df['time'] = time
test2_df['y'] = data
test2_df['meter_id'] = meter_id
test2_df.to_csv(path + "input/test_df.csv",index=True)